## Basic Self Attention

Basic self attention
- calculate dot product bt embeddings for each token in sequence and query (selected token at this time)
- normalize this to basically calculate attention scores of each input token
- multiply token embedding and attention score to get attention weights


In [8]:
import torch

input_embeddings = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

use extremely low dimension embeddings for illustration

In [9]:
query = input_embeddings[1] # journey
attention_scores_2 = torch.empty(input_embeddings.shape[0])
for i, token_embedding in enumerate(input_embeddings):
    attention_scores_2[i] = torch.dot(query, token_embedding)
print(attention_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [10]:
#simple normalization
attention_weights_2 = attention_scores_2/attention_scores_2.sum()
print(attention_weights_2)

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])


In [11]:
#normalization with softmax
attention_weights_2 = torch.softmax(attention_scores_2, dim=0)
print(attention_weights_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [12]:
context_vector_2 = torch.zeros(query.shape[0])
for i, token_embedding in enumerate(input_embeddings):
    context_vector_2 += attention_weights_2[i] * token_embedding
print(context_vector_2)

tensor([0.4419, 0.6515, 0.5683])


### Calculate attention weights for all input tokens

In [13]:
#naive
attention_scores = torch.empty(6, 6)
for i, query in enumerate(input_embeddings):
    for j, key in enumerate(input_embeddings):
        attention_scores[i, j] = torch.dot(query, key)
print(attention_scores)


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [14]:
#vectorized
attention_scores = input_embeddings @ input_embeddings.T
print(attention_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [15]:
attention_weights = torch.softmax(attention_scores, dim=-1)
print(attention_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [16]:
all_context_vectors = attention_weights @ input_embeddings
print(all_context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self Attention w/ Trainable Weights

now we change query, key, and value into separate trainable weight matrices 

In [17]:
#single example
x_2 = input_embeddings[1]
dim_in = input_embeddings.shape[1]
dim_out = 2

#initialize weight matrices
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)

query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [18]:
#calculate all keys and values wrt query 2
keys = input_embeddings @ W_key
values = input_embeddings @ W_value
print(keys.shape)
print(values.shape)


torch.Size([6, 2])
torch.Size([6, 2])


In [19]:
#attention score for query 2 key 2
keys_2 = keys[1]
attention_score_22 = torch.dot(query_2, keys_2)
print(attention_score_22)

tensor(1.8524)


In [20]:
#all attention scores for query 2
attention_scores_2 = query_2 @ keys.T
print(attention_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


now when we normalize to get attention weights, we also want to scale by the square root of the embedding dimensions of keys matrix. this is why it is called *scaled* dot product attention

In [21]:
dim_keys = keys.shape[-1]
attention_weights_2 = torch.softmax(attention_scores_2 / (dim_keys ** 0.5), dim=-1)
print(attention_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [22]:
context_vector_2 = attention_weights_2 @ values
print(context_vector_2)

tensor([0.3061, 0.8210])


In [23]:
import torch.nn as nn

class SelfAttentionV2(nn.Module):
    def __init__(self, dim_in, dim_out, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)

    def forward(self, x):
        query = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_scores = query @ keys.T
        attention_weights = torch.softmax(attention_scores / (dim_keys ** 0.5), dim=-1)
        context_vector = attention_weights @ values
        return context_vector

In [24]:
sa_v2 = SelfAttentionV2(dim_in, dim_out)
context_vector = sa_v2(input_embeddings)
print(context_vector)

tensor([[0.5085, 0.3508],
        [0.5084, 0.3508],
        [0.5084, 0.3506],
        [0.5074, 0.3471],
        [0.5076, 0.3446],
        [0.5077, 0.3493]], grad_fn=<MmBackward0>)


## Causal Attention

mask future tokens

In [25]:
queries = sa_v2.W_query(input_embeddings)
keys = sa_v2.W_key(input_embeddings) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)


context_length = attention_weights.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
mask_simple

tensor([[0.1362, 0.1730, 0.1736, 0.1713, 0.1792, 0.1666],
        [0.1359, 0.1730, 0.1735, 0.1716, 0.1790, 0.1670],
        [0.1366, 0.1729, 0.1734, 0.1714, 0.1788, 0.1669],
        [0.1493, 0.1701, 0.1704, 0.1697, 0.1732, 0.1674],
        [0.1589, 0.1690, 0.1692, 0.1667, 0.1712, 0.1649],
        [0.1408, 0.1715, 0.1718, 0.1717, 0.1758, 0.1684]],
       grad_fn=<SoftmaxBackward0>)


tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [26]:
masked_attention_weights = attention_weights * mask_simple
masked_attention_weights

tensor([[0.2098, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1385, 0.2379, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1390, 0.2369, 0.2326, 0.0000, 0.0000, 0.0000],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.0000, 0.0000],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.0000],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [27]:
row_sums = masked_attention_weights.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_attention_weights / row_sums
masked_simple_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3680, 0.6320, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2284, 0.3893, 0.3822, 0.0000, 0.0000, 0.0000],
        [0.2046, 0.2956, 0.2915, 0.2084, 0.0000, 0.0000],
        [0.1753, 0.2250, 0.2269, 0.1570, 0.2158, 0.0000],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

instead of calculating attention weights and then masking, it is more efficient to mask w/ -inf before softmax

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attention_weights.masked_fill(mask == 1, float('-inf'))
masked

tensor([[0.2098,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.1385, 0.2379,   -inf,   -inf,   -inf,   -inf],
        [0.1390, 0.2369, 0.2326,   -inf,   -inf,   -inf],
        [0.1435, 0.2074, 0.2046, 0.1462,   -inf,   -inf],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879,   -inf],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [31]:
attention_weights = torch.softmax(masked / keys.shape[-1] ** .5, dim=-1)
attention_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4824, 0.5176, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3185, 0.3413, 0.3402, 0.0000, 0.0000, 0.0000],
        [0.2444, 0.2557, 0.2551, 0.2448, 0.0000, 0.0000],
        [0.1970, 0.2031, 0.2033, 0.1947, 0.2019, 0.0000],
        [0.1633, 0.1728, 0.1721, 0.1637, 0.1588, 0.1693]])

add dropout

In [32]:
torch.manual_seed(123)
dropout = nn.Dropout(p=0.5)
example = torch.ones(6, 6)

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [33]:
torch.manual_seed(123)
print(dropout(attention_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6369, 0.6826, 0.6805, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5113, 0.5103, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4061, 0.0000, 0.3895, 0.0000, 0.0000],
        [0.0000, 0.3456, 0.3442, 0.3274, 0.3176, 0.0000]])


In [35]:
batch = torch.stack((input_embeddings, input_embeddings), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [36]:
class CausalAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.W_query = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        query = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attention_scores = query @ keys.transpose(1, 2)
        attention_scores.masked_fill(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        
        attention_weights = torch.softmax(attention_scores / (keys.shape[-1] ** 0.5), dim=-1)
        attention_weights = self.dropout(attention_weights)

        context_vector = attention_weights @ values
        return context_vector


In [38]:
torch.manual_seed(123)
context_length = batch.shape[1]
causal_attention = CausalAttention(dim_in, dim_out, context_length, 0.0)
context_vectors = causal_attention(batch)
print(context_vectors)

print(context_vectors.shape)


tensor([[[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]],

        [[-0.5337, -0.1051],
         [-0.5323, -0.1080],
         [-0.5323, -0.1079],
         [-0.5297, -0.1076],
         [-0.5311, -0.1066],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
torch.Size([2, 6, 2])


# Multi-head attention

In [43]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([
            CausalAttention(dim_in, dim_out, context_length, dropout, qkv_bias)
            for _ in range(num_heads)
        ])
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [44]:
torch.manual_seed(123)
context_length = batch.shape[1]
dim_in, dim_out = 3, 2
mha = MultiHeadAttentionWrapper(dim_in, dim_out, context_length, 0.0, 2)
context_vectors = mha(batch)
print(context_vectors)
print(context_vectors.shape)

tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 4])


refactor to process heads in parallel. do this by reshaping input into multiple heads and combining results after computing attention

In [45]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim_in, dim_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert dim_out % num_heads == 0
        self.dim_out = dim_out
        self.num_heads = num_heads
        self.head_dim = dim_out // num_heads
        self.W_query = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.out_projection = torch.nn.Linear(dim_out, dim_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
    def forward(self, x):
        b, num_tokens, dim_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attention_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attention_scores.masked_fill_(mask_bool, -torch.inf)

        attention_weights = torch.softmax(attention_scores / (keys.shape[-1] ** 0.5), dim=-1)
        attention_weights = self.dropout(attention_weights)
        context_vector = (attention_weights @ values).transpose(1, 2)
        context_vector = context_vector.contiguous().view(b, num_tokens, self.dim_out)
        return self.out_projection(context_vector)